In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to project
%cd /content/drive/MyDrive/Colab\ Notebooks/italian_teacher

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/italian_teacher


In [2]:
# Install dependencies
!pip install -q transformers trl accelerate peft datasets spacy sentence-transformers bitsandbytes
!python -m spacy download it_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 126.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:


import json
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOConfig, GRPOTrainer
from datasets import Dataset
from getpass import getpass

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Set OpenAI API key for full professional-grade reward function validation
if "OPENAI_API_KEY" not in os.environ:
    OPENAI_API_KEY = getpass("Enter OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ OpenAI API enabled - Full professional validation (spaCy + LLM)")
print("   - Article validation: LLM")
print("   - Grammar backup: LLM for scores < 8.0")
print("   - Coherence checking: LLM")

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: NVIDIA L4
Enter OpenAI API key: ··········
✅ OpenAI API enabled - Full professional validation (spaCy + LLM)
   - Article validation: LLM
   - Grammar backup: LLM for scores < 8.0
   - Coherence checking: LLM


In [4]:
from src.rl.reward_function import ExerciseRewardFunction

# Initialize reward function (loads vocabulary + models)
print("Loading reward function...")
reward_fn = ExerciseRewardFunction()
print("✅ Reward function ready")

Loading reward function...
Loading spaCy model: it_core_news_sm...
✅ spaCy model loaded
Reward function will use device: cuda
Initializing scorers...
Pre-loading CEFR vocabulary (16,887 words)...
✅ Loaded 16887 Italian words from vocabulary list
✅ Loaded vocabulary for all CEFR levels
  ✅ OpenAI validation enabled for B2+ levels
  ✅ LLM fluency checking enabled (OpenAI API)
  ✅ LLM grammar checking enabled (OpenAI API)
Loading sentence transformer for topic similarity...
✅ Sentence transformer loaded in cuda
  ✅ LLM topic checking enabled (OpenAI API)
  ✅ LLM coherence checking enabled (OpenAI API)
✅ Reward function initialized with 8 professional scorers
✅ Reward function ready


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load your Round 1 GRPO-trained model for evaluation
MODEL_PATH = "./models/italian_v8_grpo_round2"  # ← Round 1 GRPO model (Oct 17)

print(f"Loading model from: {MODEL_PATH}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    load_in_8bit=True
)
model.eval()

print(f"✅ Model loaded on device: {next(model.parameters()).device}")

Loading model from: ./models/italian_v8_grpo_round2


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model loaded on device: cuda:0


In [6]:
import os

# Load pre-generated validation requests
if os.path.exists("src/rl/validation_requestss.json"):
    print("Loading existing validation requests...")
    with open("src/rl/validation_requestss.json", "r") as f:
        validation_requests = json.load(f)
else:
    # If not exists, generate them
    from src.rl.generate_training_requests import generate_training_requests
    print("Generating new validation requests...")
    validation_requests = generate_training_requests(
        num_requests=20,
        output_path="src/rl/validation_requests.json"
    )

print(f"✅ Loaded {len(validation_requests)} validation requests")

Generating new validation requests...
Loading vocabulary...
Generating 20 training requests...
✅ Saved 20 requests to src/rl/validation_requests.json

📊 Request Statistics:
By CEFR level:
  A1: 1
  A2: 6
  B1: 5
  B2: 4
  C1: 3
  C2: 1

Top 5 grammar focuses:
  future_tense: 7
  past_tense: 3
  imperfect_tense: 3
  present_tense: 2
  subjunctive: 2
✅ Loaded 20 validation requests


In [7]:
from src.rl.iterative_training import evaluate_model_on_requests, create_round2_dataset
# After Round 1 training:
low, high = evaluate_model_on_requests(model, tokenizer, reward_fn, validation_requests, batch_size=50, output_path='eval.json')


✅ Model is already quantized and mapped to devices.
✅ SentenceTransformer moved to CUDA for topic scoring.
📊 Evaluating 20 requests in batches of 50 (max_new_tokens=400)
Model device: cuda:0


Evaluating Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/content/drive/MyDrive/Colab Notebooks/italian_teacher/src/rl/iterative_training.py:326: RuntimeWarning: coroutine 'ExerciseRewardFunction.score' was never awaited
  score, breakdown = reward_fn.score(exercise, request)



✅ Evaluation complete. Processed: 20
  Low scoring (<92): 20
  High scoring (>=92): 0
  Avg score (processed): 10.00
💾 Final results saved to eval.json


In [8]:
# Summary of evaluation results
print("="*80)
print("📊 GRPO MODEL EVALUATION SUMMARY")
print("="*80)

print(f"\n✅ Evaluated {len(validation_requests)} validation examples")
print(f"   Low scoring (<92): {len(low)} examples")
print(f"   High scoring (≥92): {len(high)} examples")

if low:
    low_scores = [item['score'] for item in low]
    print(f"\n📉 Low scoring stats:")
    print(f"   Average: {sum(low_scores)/len(low_scores):.2f}")
    print(f"   Min: {min(low_scores):.2f}")
    print(f"   Max: {max(low_scores):.2f}")

if high:
    high_scores = [item['score'] for item in high]
    print(f"\n📈 High scoring stats:")
    print(f"   Average: {sum(high_scores)/len(high_scores):.2f}")
    print(f"   Min: {min(high_scores):.2f}")
    print(f"   Max: {max(high_scores):.2f}")

all_scores = [item['score'] for item in low + high]
overall_avg = sum(all_scores) / len(all_scores)

print(f"\n🎯 Overall average score: {overall_avg:.2f}/100")
print(f"\n💾 Detailed results saved to: eval.json")

print("\n" + "="*80)
print("📋 NEXT STEPS:")
print("="*80)
if overall_avg >= 85:
    print("✅ Model quality is GOOD (≥85) - Ready to deploy!")
    print("   Consider this model for production use.")
elif overall_avg >= 75:
    print("⚠️  Model quality is OK (75-85) - Can deploy or do another round")
    print("   If you want higher quality, consider training another round.")
else:
    print("❌ Model quality needs improvement (<75)")
    print("   Recommend another training round with adjusted hyperparameters.")

print("\n📊 To view sample outputs:")
print("   - Check eval.json for detailed per-exercise scores")
print("   - Low scoring examples can be used for next training round")
print("="*80)

📊 GRPO MODEL EVALUATION SUMMARY

✅ Evaluated 20 validation examples
   Low scoring (<92): 20 examples
   High scoring (≥92): 0 examples

📉 Low scoring stats:
   Average: 10.00
   Min: 10.00
   Max: 10.00

🎯 Overall average score: 10.00/100

💾 Detailed results saved to: eval.json

📋 NEXT STEPS:
❌ Model quality needs improvement (<75)
   Recommend another training round with adjusted hyperparameters.

📊 To view sample outputs:
   - Check eval.json for detailed per-exercise scores
   - Low scoring examples can be used for next training round
